# Logical inferences

**Prerequisites: [Logic](TODO)**

An agent knows a statement $S$ is true if and only if:
* $S$ is true
* It believes $S$ is true
* It is justified in believing that $S$ is true

We can formalized this as [propositional logic](TODO)



We have the following definitions:

**Knowledge base**: A list of things that the agent knows to be true, _ie_ [premise](TODO)

Hence, the problem is that given a knowledge base and some assertion $\varphi$, how would an agent determine the truthfulness of said assertion?


This is basically a [SAT](../algorithm_analysis/np_hardness.ipynb#SAT) problem, and the straightforward way would be to find a assignment $\tau$ that satisfy $\varphi$.
Symbolically, we are finding the existence/non-existence of a $\tau$ such that:
$$
\tau \models \varphi
$$

For example, given $\varphi = \left( \left( a \lor b \right) \land  \lnot c \right)$
We can see that $\tau_1 = \{a = 1, b = 1, c = 1 \}$ results in $\varphi(\tau_1) = 0$.

While $\tau_2 = \{a = 1, b = 0, c = 0 \}$ results in $\varphi(\tau_2) = 1$.

Hence, we make the following definitions about $\varphi$:

$\varphi$ is **SAT** (satisfiable) if 
$$
\exists \tau: \tau \models \varphi
$$

$\varphi$ is **VALID** if 
$$
\forall\tau: \tau \models \varphi
$$

$\varphi$ is **UNSAT** (unsatisfiable) if 
$$
\forall\tau: \tau \not \models \varphi
$$

For example, $(a \lor \lnot a)$ is VALID and SAT.
$(a \lor b)$ is SAT.
And $(a \land \lnot a)$ is UNSAT.

Also, we define the following relationship between clauses:

If and only if $\varphi \rightarrow \psi$ is VALID, then:
$$
\varphi \models \psi
$$

If and only if $\varphi  \leftrightarrow \psi$ is VALID, then:
$$
\varphi \quad |\hspace{-.4em} == \hspace{-.4em}| \quad \psi
$$
This is saying that the two are **semantically equivalent**.

<span hidden> TODO: Check if can link with logic </span>

## Conjunction normal form

We define the conjunction normal form as a formula $\varphi$ in the form of
$$
C_1 \land C_2 \land \dots \land C_n
$$

and $C_i$ is in the form of
$$
C_i = l_1 \lor l_2 \lor \dots \lor l_{k_i}
$$
where $l$ are the literals, which are either $a$ or $\lnot a$, for $a \in PROP$

### Theorem
Every formula $\varphi$ can be converted into a CNF

This process usually involves application of De Morgan's rule and associativity/communitivity of boolean operators.

For ease of discussion, we will be referring to the CNF form from now on for consistency.

With this background, we will now proceed to prove the validity of some statement $\varphi$.

We are given some $KB$, and which to determine if $KB \models \varphi$. ($\models$ means "entails" in this context, please be careful between the type of variables used with this operator as the meaning is context sensitive).

That is, we wish to prove if $KB \rightarrow \varphi$ is VALID.

Notice that we can perform a [proof by contraction](TODO). 
If we prove that $\lnot \varphi$ is UNSAT, then the proof is complete.

## Naive approach
A simply way to prove this is to check every $\tau$ possible, and find that there exists an satisfiable assignment to $\lnot \varphi$, or find there no such assignment exists.
This approach is $O(2^{|PROP|}$), which is rather slow.

## Resolution

Suppose we have the following formula:
$$
\varphi = p_1 \land \lnot p_1 \land \dots
$$

Notice that since there is a $(p_1 \land \lnot p_1)$, this is never satisfiable, and thus would be UNSAT.
And we didn't need to consider the rest of the formula.

We symbolize the resolution step as follows:
$$
\frac{(p) \land (\lnot p)}{\square}
$$
where $\square$ is the unsatisiable set

Another logical inference is as follows:
$$
\frac{(a \lor b) \land (\lnot b \lor c)}{a \land c}
$$

And with this, we can prove our assertions programmatically.
We just keep generating new clauses using our previous clauses until we find an unsatisiable set.
If we reach a point where no new clauses can be generated, and no unsastifiable set is generated, then $\lnot \varphi$ must be SAT and hence $KB \rightarrow \varphi$ must be INVALID.

In [121]:
from dataclasses import dataclass


@dataclass(eq=True, frozen=True)
class Literal:
    symbol: str
    is_inverted: bool = False

    def invert(self):
        return Literal(self.symbol, not self.is_inverted)


"""A disjunctive clause"""


class Clause:
    def __init__(self, *literals):
        self.literals = set(literals)


class KnowledgeBase:
    def __init__(self, *clauses):
        self.clauses = clauses

    def check_valid(self, clause):
        kb = self.clauses.add(clause)
        while True:
            old_kb = kb.copy()
            if is_unsat(phi):
                return False

            for a in phi:
                for b in phi:
                    if a != b:
                        clauses = resolve(a, b)
                        for c in clauses:
                            if c not in phi:
                                phi.append(c)

            if old_phi == phi:
                return True

In [120]:
Clause(Literal("a"), Literal("b"), Literal("c", True), Literal("a")).literals

{Literal(symbol='a', is_inverted=False),
 Literal(symbol='b', is_inverted=False),
 Literal(symbol='c', is_inverted=True)}

In [95]:
def invert(lit):
    return "!" + lit if "!" not in lit else lit[1:]


def resolve(a, b):
    literals = [lit for lit in a]
    inv_lit = [invert(lit) for lit in a]

    result = []
    for lit in b:
        if lit in inv_lit:
            new_set = a.union(b)
            new_set.remove(lit)
            new_set.remove(invert(lit))
            result.append(new_set)
    return result


def is_unsat(form):
    singular_clauses = set(min(clause) for clause in form if len(clause) == 1)
    for clause in singular_clauses:
        if invert(clause) in singular_clauses:
            return True

    return False

In [126]:
def check_sat(phi):
    while True:
        old_phi = phi.copy()
        if is_unsat(phi):
            return False

        for a in phi:
            for b in phi:
                if a != b:
                    clauses = resolve(a, b)
                    for c in clauses:
                        if c not in phi:
                            phi.append(c)

        if old_phi == phi:
            return True

In [127]:
phi = [
    {"a", "b", "c"},
    {"!a", "b", "d"},
    {"c", "!d"},
    {"!b", "c"},
    {"!b", "!c"},
    {"b", "!c"}
]

In [128]:
check_sat(phi)

False

In [102]:
phi = [{"a", "b"}, {"a", "!b"}]

In [103]:
check_sat(phi)

True